In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reviews.csv to reviews.csv


In [ ]:
df = pd.read_csv("reviews.csv")

In [ ]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,35644,3024649,2012-12-04,5394983,Anne,Yvette picked us up from the bus stop on our a...
1,35644,3040421,2012-12-08,3712269,Catherine,"Lovely house with beautiful views, close to th..."
2,35644,3126050,2012-12-20,2908896,Paul,Yvette is a generous host and a georgeous pers...
3,35644,3142082,2012-12-23,3941752,Carmel,This is our first experience with Air B & B; a...
4,35644,3194549,2012-12-31,2735578,Katharina,Yvette is a very friendly host and happy to gi...


In [ ]:
from collections import Counter
import re

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
df.isna().sum()

listing_id         0
id                 0
date               0
reviewer_id        0
reviewer_name      0
comments         114
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.isna().sum()

listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64

In [ ]:
df.info

<bound method DataFrame.info of         listing_id  ...                                           comments
0            35644  ...  Yvette picked us up from the bus stop on our a...
1            35644  ...  Lovely house with beautiful views, close to th...
2            35644  ...  Yvette is a generous host and a georgeous pers...
3            35644  ...  This is our first experience with Air B & B; a...
4            35644  ...  Yvette is a very friendly host and happy to gi...
...            ...  ...                                                ...
313384    42642623  ...  Rays place is in a beautiful & historic area o...
313385    42642623  ...  Ray was super helpful and thoughtful. The room...
313386    42669236  ...  Great place, location and a good value! \nKitc...
313387    42763471  ...  Fantastic.\nLocation, right in the heart of de...
313388    42795957  ...  Great location really nice place to stay nice ...

[313275 rows x 6 columns]>

In [ ]:
df = df.reset_index()

In [ ]:
df['index'] = df.index

In [ ]:
df.head()

,index,listing_id,id,date,reviewer_id,reviewer_name,comments
0,0,35644,3024649,2012-12-04,5394983,Anne,Yvette picked us up from the bus stop on our a...
1,1,35644,3040421,2012-12-08,3712269,Catherine,"Lovely house with beautiful views, close to th..."
2,2,35644,3126050,2012-12-20,2908896,Paul,Yvette is a generous host and a georgeous pers...
3,3,35644,3142082,2012-12-23,3941752,Carmel,This is our first experience with Air B & B; a...
4,4,35644,3194549,2012-12-31,2735578,Katharina,Yvette is a very friendly host and happy to gi...


In [ ]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['comments'])

In [ ]:
#nlp = spacy.load('en', disable=['ner', 'parser'])
def cleaning(doc):
    txt = [token.text for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [ ]:
from time import time  # To time our operations
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 20.67 mins


In [ ]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
print(df_clean.shape)
print('\n')
print(df_clean.head())

(294533, 1)


                                               clean
0  yvette picked bus stop arrival hobart immediat...
1  lovely house beautiful views close city centre...
2  yvette generous host georgeous person feel wel...
3  experience air b b excellent experience yvette...
4  yvette friendly host happy lots advice enjoyed...


In [ ]:
text = []
label = []
explain = []

In [ ]:
t = time()
for doc in nlp.pipe(df['comments'], disable=["tagger", "parser"], batch_size=5000, n_threads=-1):
    # Do something with the doc here
    tt = [x.text for x in doc.ents]
    text.extend(tt)
    l = [x.label_ for x in doc.ents]
    label.extend(l)
    e = [str(spacy.explain(x.label_)) for x in doc.ents]
    explain.extend(e)

print('Time to process everything: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
ner_df = pd.DataFrame(
    {'Word': text,
     'Label': label,
     'Description': explain
    })

In [ ]:
ner_df.head()

,Word,Label,Description
0,Yvette,PERSON,"People, including fictional"
1,Hobart,LOC,"Non-GPE locations, mountain ranges, bodies of ..."
2,Mr Blue,PERSON,"People, including fictional"
3,Yvette,PERSON,"People, including fictional"
4,Yvette,PERSON,"People, including fictional"


In [ ]:
len(ner_df[(ner_df['Word'] == 'Hobart') & (ner_df['Label'] == 'GPE')])

15083

In [ ]:
Counter(list_items).most_common(20)

[('Hobart - LOC - Non-GPE locations, mountain ranges, bodies of water', 15083),
 ('Hobart - GPE - Countries, cities, states', 12913),
 ('Launceston - GPE - Countries, cities, states', 10469),
 ('Tasmania - GPE - Countries, cities, states', 8480),
 ('two - CARDINAL - Numerals that do not fall under another type', 6386),
 ('Salamanca - GPE - Countries, cities, states', 5005),
 ('Hobart - PERSON - People, including fictional', 4968),
 ('one - CARDINAL - Numerals that do not fall under another type', 4427),
 ('2 - CARDINAL - Numerals that do not fall under another type', 4228),
 ('first - ORDINAL - "first", "second", etc.', 4161),
 ('Hobart - ORG - Companies, agencies, institutions, etc.', 4020),
 ('Tassie - GPE - Countries, cities, states', 3948),
 ('CBD - GPE - Countries, cities, states', 3940),
 ('night - TIME - Times smaller than a day', 3236),
 ('5 - CARDINAL - Numerals that do not fall under another type', 2786),
 ('one night - TIME - Times smaller than a day', 2589),
 ('John - PERSO

Collocations with NLTK

In [ ]:
import nltk.collocations
import nltk.corpus
import collections
import spacy
from collections import Counter
nlp = spacy.load('en_core_web_lg')

In [ ]:
#No l'he utilitzat

t = time()
tok_words = []
df_len = len(df)
for i in range(df_len):
  doc = df['comments'][i].lower()
  doc = nlp(doc)
  words = [token.lema for token in doc if token.is_stop != True and token.is_punct != True and token.is_digit != True]
  tok_words = tok_words + words

print('Time to process everything: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
def clean_comments(text):
    tok_words = []
    #remove punctuations
    doc = df['comments'][i].lower()
    doc = nlp(doc)
    words = [token.lemma_ for token in doc if token.is_stop != True and token.is_punct != True and token.is_digit != True]
    tok_words = tok_words + words
    return tok_words

In [ ]:
t = time()

#apply function to clean and lemmatize comments
lemmatized = df['comments'].map(clean_comments)

#make sure to lowercase everything
lemmatized = lemmatized.map(lambda x: [word.lower() for word in x])

#turn all comments' tokens into one single list
unlist_comments = [item for items in lemmatized for item in items]

print('Time to process everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to process everything: 108.33 mins


In [ ]:
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

In [ ]:
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_comments)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_comments)

In [ ]:
bigram_freq = bigramFinder.ngram_fd.items()

In [ ]:
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)

In [ ]:
bigramFreqTable.head().reset_index(drop=True)

,bigram,freq
0,"(place, gorgeous)",313275
1,"(gorgeous, view)",313275
2,"(amazing, rock)",313275
3,"(rock, pool)",313275
4,"(pool, beach)",313275


In [ ]:
bigramFreqTable[:10]

,bigram,freq
0,"(place, gorgeous)",313275
1,"(gorgeous, view)",313275
22,"(amazing, rock)",313275
23,"(rock, pool)",313275
24,"(pool, beach)",313275
25,"(beach, hill)",313275
26,"(hill, want)",313275
27,"(want, experience)",313275
28,"(experience, bay)",313275
29,"(bay, fire)",313275


In [ ]:
#Ja he tret les stop words a la funcio clean_comments
#get english stopwords
en_stopwords = ['\n\n']#set(stopwords.words('english'))

In [ ]:
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords:
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

In [ ]:
#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filtered_bi[:10]

,bigram,freq
1,"(gorgeous, view)",313275
23,"(rock, pool)",313275
24,"(pool, beach)",313275
25,"(beach, hill)",313275
27,"(want, experience)",313275
28,"(experience, bay)",313275
29,"(bay, fire)",313275
30,"(fire, stay)",313275
34,"(fantastic, price)",313275
35,"(price, reasonable)",313275


In [ ]:
def rightTypesTri(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or '  ' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords:
            return False
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[2][1] in third_type:
        return True
    else:
        return False

In [ ]:
trigram_freq = trigramFinder.ngram_fd.items()

In [ ]:
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)

In [ ]:
trigramFreqTable.head().reset_index(drop=True)

,trigram,freq
0,"(place, gorgeous, view)",313275
1,"(bay, fire, stay)",313275
2,"(amazing, rock, pool)",313275
3,"(rock, pool, beach)",313275
4,"(pool, beach, hill)",313275


In [ ]:
trigramFreqTable[:10]

,trigram,freq
0,"(place, gorgeous, view)",313275
29,"(bay, fire, stay)",313275
22,"(amazing, rock, pool)",313275
23,"(rock, pool, beach)",313275
24,"(pool, beach, hill)",313275
25,"(beach, hill, want)",313275
26,"(hill, want, experience)",313275
27,"(want, experience, bay)",313275
28,"(experience, bay, fire)",313275
30,"(fire, stay, \n\n)",313275


In [ ]:
filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda x: rightTypesTri(x))]

In [ ]:
filtered_tri[:10]

,trigram,freq
0,"(place, gorgeous, view)",313275
29,"(bay, fire, stay)",313275
23,"(rock, pool, beach)",313275
24,"(pool, beach, hill)",313275
26,"(hill, want, experience)",313275
27,"(want, experience, bay)",313275
28,"(experience, bay, fire)",313275
1,"(gorgeous, view, fantastic)",313275
33,"(host, fantastic, price)",313275
34,"(fantastic, price, reasonable)",313275


In [ ]:
freq_bi = filtered_bi[:20].bigram.values

In [ ]:
freq_tri = filtered_tri[:20].trigram.values

PMI

In [ ]:
bigramFinder.apply_freq_filter(20)

In [ ]:
bigramPMITable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.pmi)), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)

In [ ]:
bigramPMITable[bigramPMITable['bigram'] != '\n\n']

In [ ]:
bigramPMITable[:10]

,bigram,PMI
0,"(\n\n, host)",5.321928
15,"(lovely, enjoy)",5.321928
27,"(want, experience)",5.321928
26,"(uk, desperate)",5.321928
25,"(stay, \n\n)",5.321928
24,"(room, game)",5.321928
23,"(rock, pool)",5.321928
22,"(restaurant, nearby)",5.321928
21,"(reasonable, uk)",5.321928
1,"(amazing, rock)",5.321928


In [ ]:
filteredPMI_bi = bigramPMITable[bigramPMITable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filteredPMI_bi[:10]

,bigram,PMI
27,"(want, experience)",5.321928
26,"(uk, desperate)",5.321928
24,"(room, game)",5.321928
23,"(rock, pool)",5.321928
22,"(restaurant, nearby)",5.321928
21,"(reasonable, uk)",5.321928
19,"(pool, beach)",5.321928
17,"(nice, restaurant)",5.321928
20,"(price, reasonable)",5.321928
13,"(house, amenity)",5.321928


In [ ]:
trigramFinder.apply_freq_filter(20)

In [ ]:
trigramPMITable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.pmi)), columns=['trigram','PMI']).sort_values(by='PMI', ascending=False)

In [ ]:
trigramPMITable[:10]

,trigram,PMI
0,"(amazing, rock, pool)",10.643856
12,"(nearby, amazing, rock)",10.643856
22,"(want, experience, bay)",10.643856
21,"(uk, desperate, return)",10.643856
1,"(amenity, layout, lovely)",10.643856
19,"(room, game, dvd)",10.643856
18,"(rock, pool, beach)",10.643856
17,"(restaurant, nearby, amazing)",10.643856
16,"(reasonable, uk, desperate)",10.643856
15,"(price, reasonable, uk)",10.643856


In [ ]:
filteredPMI_tri = trigramPMITable[trigramPMITable.trigram.map(lambda x: rightTypesTri(x))]

In [ ]:
filteredPMI_tri[:10]

,trigram,PMI
22,"(want, experience, bay)",10.643856
21,"(uk, desperate, return)",10.643856
19,"(room, game, dvd)",10.643856
18,"(rock, pool, beach)",10.643856
17,"(restaurant, nearby, amazing)",10.643856
16,"(reasonable, uk, desperate)",10.643856
15,"(price, reasonable, uk)",10.643856
14,"(pool, beach, hill)",10.643856
13,"(nice, restaurant, nearby)",10.643856
5,"(experience, bay, fire)",10.643856


In [ ]:
pmi_bi = filteredPMI_bi[:20].bigram.values

In [ ]:
pmi_tri = filteredPMI_tri[:20].trigram.values

T-Test

In [ ]:
bigramTtable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.student_t)), columns=['bigram','t']).sort_values(by='t', ascending=False)

In [ ]:
bigramTtable.head()

,bigram,t
0,"(\n\n, host)",545.717003
15,"(lovely, enjoy)",545.717003
27,"(want, experience)",545.717003
26,"(uk, desperate)",545.717003
25,"(stay, \n\n)",545.717003


In [ ]:
filteredT_bi = bigramTtable[bigramTtable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filteredT_bi[:10]

,bigram,t
27,"(want, experience)",545.717003
26,"(uk, desperate)",545.717003
24,"(room, game)",545.717003
23,"(rock, pool)",545.717003
22,"(restaurant, nearby)",545.717003
21,"(reasonable, uk)",545.717003
19,"(pool, beach)",545.717003
17,"(nice, restaurant)",545.717003
20,"(price, reasonable)",545.717003
13,"(house, amenity)",545.717003


In [ ]:
trigramTtable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.student_t)), columns=['trigram','t']).sort_values(by='t', ascending=False)

In [ ]:
trigramTtable.head()

,trigram,t
0,"(amazing, rock, pool)",559.359928
12,"(nearby, amazing, rock)",559.359928
22,"(want, experience, bay)",559.359928
21,"(uk, desperate, return)",559.359928
1,"(amenity, layout, lovely)",559.359928


In [ ]:
filteredT_tri = trigramTtable[trigramTtable.trigram.map(lambda x: rightTypesTri(x))]

In [ ]:
filteredT_tri.head(10)

,trigram,t
22,"(want, experience, bay)",559.359928
21,"(uk, desperate, return)",559.359928
19,"(room, game, dvd)",559.359928
18,"(rock, pool, beach)",559.359928
17,"(restaurant, nearby, amazing)",559.359928
16,"(reasonable, uk, desperate)",559.359928
15,"(price, reasonable, uk)",559.359928
14,"(pool, beach, hill)",559.359928
13,"(nice, restaurant, nearby)",559.359928
5,"(experience, bay, fire)",559.359928


In [ ]:
t_bi = filteredT_bi[:20].bigram.values

In [ ]:
t_tri = filteredT_tri[:20].trigram.values

Chi-Square

In [ ]:
bigramChiTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','chi-sq']).sort_values(by='chi-sq', ascending=False)

In [ ]:
bigramChiTable.head(10)

,bigram,chi-sq
0,"(\n\n, host)",12531000.0
15,"(lovely, enjoy)",12531000.0
27,"(want, experience)",12531000.0
26,"(uk, desperate)",12531000.0
25,"(stay, \n\n)",12531000.0
24,"(room, game)",12531000.0
23,"(rock, pool)",12531000.0
22,"(restaurant, nearby)",12531000.0
21,"(reasonable, uk)",12531000.0
1,"(amazing, rock)",12531000.0


In [ ]:
filteredCHI_bi = bigramChiTable[bigramChiTable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filteredCHI_bi[:10]

,bigram,chi-sq
27,"(want, experience)",12531000.0
26,"(uk, desperate)",12531000.0
24,"(room, game)",12531000.0
23,"(rock, pool)",12531000.0
22,"(restaurant, nearby)",12531000.0
21,"(reasonable, uk)",12531000.0
19,"(pool, beach)",12531000.0
17,"(nice, restaurant)",12531000.0
20,"(price, reasonable)",12531000.0
13,"(house, amenity)",12531000.0


In [ ]:
trigramChiTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.chi_sq)), columns=['trigram','chi-sq']).sort_values(by='chi-sq', ascending=False)

In [ ]:
trigramChiTable.head(10)

,trigram,chi-sq
0,"(amazing, rock, pool)",5.015613e+08
12,"(nearby, amazing, rock)",5.015613e+08
22,"(want, experience, bay)",5.015613e+08
21,"(uk, desperate, return)",5.015613e+08
1,"(amenity, layout, lovely)",5.015613e+08
19,"(room, game, dvd)",5.015613e+08
18,"(rock, pool, beach)",5.015613e+08
17,"(restaurant, nearby, amazing)",5.015613e+08
16,"(reasonable, uk, desperate)",5.015613e+08
15,"(price, reasonable, uk)",5.015613e+08


In [ ]:
filteredCHI_tri = trigramChiTable[trigramChiTable.trigram.map(lambda x: rightTypesTri(x))]

In [ ]:
chi_bi = filteredCHI_bi[:20].bigram.values

chi_tri = filteredCHI_tri[:20].trigram.values

Likelihood

In [ ]:
bigramLikTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.likelihood_ratio)), columns=['bigram','likelihood ratio']).sort_values(by='likelihood ratio', ascending=False)

In [ ]:
bigramLikTable.head()

,bigram,likelihood ratio
0,"(\n\n, host)",2.929919e+06
15,"(lovely, enjoy)",2.929919e+06
27,"(want, experience)",2.929919e+06
26,"(uk, desperate)",2.929919e+06
25,"(stay, \n\n)",2.929919e+06


In [ ]:
filteredLik_bi = bigramLikTable[bigramLikTable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filteredLik_bi.head(10)

,bigram,likelihood ratio
27,"(want, experience)",2.929919e+06
26,"(uk, desperate)",2.929919e+06
24,"(room, game)",2.929919e+06
23,"(rock, pool)",2.929919e+06
22,"(restaurant, nearby)",2.929919e+06
21,"(reasonable, uk)",2.929919e+06
19,"(pool, beach)",2.929919e+06
17,"(nice, restaurant)",2.929919e+06
20,"(price, reasonable)",2.929919e+06
13,"(house, amenity)",2.929919e+06


In [ ]:
trigramLikTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.likelihood_ratio)), columns=['trigram','likelihood ratio']).sort_values(by='likelihood ratio', ascending=False)

In [ ]:
trigramLikTable.head()

,trigram,likelihood ratio
0,"(amazing, rock, pool)",8.789758e+06
12,"(nearby, amazing, rock)",8.789758e+06
22,"(want, experience, bay)",8.789758e+06
21,"(uk, desperate, return)",8.789758e+06
1,"(amenity, layout, lovely)",8.789758e+06


In [ ]:
filteredLik_tri = trigramLikTable[trigramLikTable.trigram.map(lambda x: rightTypesTri(x))]

In [ ]:
filteredLik_tri.head(10)

,trigram,likelihood ratio
22,"(want, experience, bay)",8.789758e+06
21,"(uk, desperate, return)",8.789758e+06
19,"(room, game, dvd)",8.789758e+06
18,"(rock, pool, beach)",8.789758e+06
17,"(restaurant, nearby, amazing)",8.789758e+06
16,"(reasonable, uk, desperate)",8.789758e+06
15,"(price, reasonable, uk)",8.789758e+06
14,"(pool, beach, hill)",8.789758e+06
13,"(nice, restaurant, nearby)",8.789758e+06
5,"(experience, bay, fire)",8.789758e+06


In [ ]:
lik_bi = filteredLik_bi[:20].bigram.values

lik_tri = filteredLik_tri[:20].trigram.values

Comparison

In [ ]:
bigramsCompare = pd.DataFrame([freq_bi, pmi_bi, t_bi, chi_bi, lik_bi]).T

In [ ]:
bigramsCompare.columns = ['Frequency', 'PMI', 'T-test', 'Chi-Sq', 'Likelihood Ratio Test']

In [ ]:
bigramsCompare

,Frequency,PMI,T-test,Chi-Sq,Likelihood Ratio Test
0,"(gorgeous, view)","(want, experience)","(want, experience)","(want, experience)","(want, experience)"
1,"(rock, pool)","(uk, desperate)","(uk, desperate)","(uk, desperate)","(uk, desperate)"
2,"(pool, beach)","(room, game)","(room, game)","(room, game)","(room, game)"
3,"(beach, hill)","(rock, pool)","(rock, pool)","(rock, pool)","(rock, pool)"
4,"(want, experience)","(restaurant, nearby)","(restaurant, nearby)","(restaurant, nearby)","(restaurant, nearby)"
5,"(experience, bay)","(reasonable, uk)","(reasonable, uk)","(reasonable, uk)","(reasonable, uk)"
6,"(bay, fire)","(pool, beach)","(pool, beach)","(pool, beach)","(pool, beach)"
7,"(fire, stay)","(nice, restaurant)","(nice, restaurant)","(nice, restaurant)","(nice, restaurant)"
8,"(fantastic, price)","(price, reasonable)","(price, reasonable)","(price, reasonable)","(price, reasonable)"
9,"(price, reasonable)","(house, amenity)","(house, amenity)","(house, amenity)","(house, amenity)"


In [ ]:
trigramsCompare = pd.DataFrame([freq_tri, pmi_tri, t_tri, chi_tri, lik_tri]).T

In [ ]:
trigramsCompare.columns = ['Frequency', 'PMI', 'T-test', 'Chi-Sq', 'Likelihood Ratio Test']

In [ ]:
trigramsCompare

,Frequency,PMI,T-test,Chi-Sq,Likelihood Ratio Test
0,"(place, gorgeous, view)","(want, experience, bay)","(want, experience, bay)","(want, experience, bay)","(want, experience, bay)"
1,"(bay, fire, stay)","(uk, desperate, return)","(uk, desperate, return)","(uk, desperate, return)","(uk, desperate, return)"
2,"(rock, pool, beach)","(room, game, dvd)","(room, game, dvd)","(room, game, dvd)","(room, game, dvd)"
3,"(pool, beach, hill)","(rock, pool, beach)","(rock, pool, beach)","(rock, pool, beach)","(rock, pool, beach)"
4,"(hill, want, experience)","(restaurant, nearby, amazing)","(restaurant, nearby, amazing)","(restaurant, nearby, amazing)","(restaurant, nearby, amazing)"
5,"(want, experience, bay)","(reasonable, uk, desperate)","(reasonable, uk, desperate)","(reasonable, uk, desperate)","(reasonable, uk, desperate)"
6,"(experience, bay, fire)","(price, reasonable, uk)","(price, reasonable, uk)","(price, reasonable, uk)","(price, reasonable, uk)"
7,"(gorgeous, view, fantastic)","(pool, beach, hill)","(pool, beach, hill)","(pool, beach, hill)","(pool, beach, hill)"
8,"(host, fantastic, price)","(nice, restaurant, nearby)","(nice, restaurant, nearby)","(nice, restaurant, nearby)","(nice, restaurant, nearby)"
9,"(fantastic, price, reasonable)","(experience, bay, fire)","(experience, bay, fire)","(experience, bay, fire)","(experience, bay, fire)"
